In [1]:
from lxml import html
import lxml
import time
import requests
import pandas as pd
import sqlite3

In [2]:
date_jour = '2017-12-31'
OUTPUT_PATH = '../_outputs/'
database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
conn = sqlite3.connect(database_file)

In [3]:
df_source = pd.read_sql_query("select * from index_chefs;", conn)
df_source.head()

,index,chef,total
0,0,florent-gouverneur,14
1,1,arnaud-bulliot,16
2,2,quentin-girerd,2
3,3,dario-jarel,12
4,4,la-belle-assiette-france,540


In [4]:
def getChefPage(chef_id):
    url = 'https://labelleassiette.fr/' + chef_id
    print("Page : " + url)
    response = requests.get(url)
    content_source = response.text
    return html.document_fromstring(content_source)

In [5]:
def get_row(doc, chef, total):
    nom = doc.xpath('//h1[@itemprop="name"]/text()')[0]
    photo = doc.xpath('//img[@class="chef-picture"]/@src')[0]
    if doc.xpath('//div[contains(@class,"chef-badge")]/span[@class="top"]/text()'):
        valide = 'OUI'
    else:
        valide = 'NON'
    texte_intro = doc.xpath('//h3[@class="chef-tagline"]/text()')[0]
    note = doc.xpath('//h4[contains(@itemprop,"ratingValue")]/@content')[0]
    avis_nb = doc.xpath('//meta[contains(@itemprop,"ratingCount")]/@content')[0]
    a_propos = doc.xpath('//div[contains(@class,"chef-card")]//span[@class="full"]/text()')[0]
    zone_ville = doc.xpath('//span[contains(@itemprop,"addressLocality")]/text()')[0]
    zone_pays_code = doc.xpath('//meta[contains(@itemprop,"addressCountry")]/@content')[0]
    if doc.xpath('//div[contains(@class,"chef-card")]/p[2]/text()'):
        langues = doc.xpath('//div[contains(@class,"chef-card")]/p[2]/text()')[0]
    else:
        langues = 'NA'
    cuisines_types = doc.xpath('//span[@itemprop="servesCuisine"]/text()') # list
    questions = doc.xpath('//div[@class="chef-question"]/p/text()')
    row = [chef, total, nom, photo, valide, texte_intro, note, avis_nb, 
           a_propos, zone_ville, zone_pays_code, langues, cuisines_types, questions]
    return row

In [6]:
columns = ['chef', 'total', 'nom', 'photo', 'valide', 'texte_intro', 'note', 'avis_nb', 
           'a_propos', 'zone_ville', 'zone_pays_code','langues', 'cuisines_types', 'questions']
df_output = pd.DataFrame(columns=columns)

In [7]:
for index, row in df_source.iterrows():
    doc = getChefPage(row['chef'])
    df_output.loc[index] = get_row(doc, row['chef'], row['total'])

Page : https://labelleassiette.fr/florent-gouverneur
Page : https://labelleassiette.fr/arnaud-bulliot
Page : https://labelleassiette.fr/quentin-girerd
Page : https://labelleassiette.fr/dario-jarel
Page : https://labelleassiette.fr/la-belle-assiette-france
Page : https://labelleassiette.fr/kevin-bense
Page : https://labelleassiette.fr/sacharanin-son-herlidou
Page : https://labelleassiette.fr/daniel-lopez
Page : https://labelleassiette.fr/raphael-courtin
Page : https://labelleassiette.fr/ella-aflalo
Page : https://labelleassiette.fr/mariana-delubac
Page : https://labelleassiette.fr/mickael-houx
Page : https://labelleassiette.fr/pierre-guichon
Page : https://labelleassiette.fr/julie-anne-charton
Page : https://labelleassiette.fr/cedric-fortes
Page : https://labelleassiette.fr/antoine-collin
Page : https://labelleassiette.fr/mojgan-tashvighi
Page : https://labelleassiette.fr/luc-fischer
Page : https://labelleassiette.fr/thomas-major
Page : https://labelleassiette.fr/hina-malik
Page : https

Page : https://labelleassiette.fr/stephane-oulevey
Page : https://labelleassiette.fr/eleonor-ill-walgener
Page : https://labelleassiette.fr/elise-deat
Page : https://labelleassiette.fr/herve-escobar
Page : https://labelleassiette.fr/sombat-et-gregor
Page : https://labelleassiette.fr/zelikha-dinga
Page : https://labelleassiette.fr/eleonora-galasso
Page : https://labelleassiette.fr/tomoko-ogata
Page : https://labelleassiette.fr/hanyen-hsu
Page : https://labelleassiette.fr/yuan-ren
Page : https://labelleassiette.fr/noemi-ito
Page : https://labelleassiette.fr/ji-hye-park
Page : https://labelleassiette.fr/mattia-salvadore
Page : https://labelleassiette.fr/jonathan-lefevre
Page : https://labelleassiette.fr/aurelie-scarabin
Page : https://labelleassiette.fr/fanny-matuszkiewicz
Page : https://labelleassiette.fr/aurelie-gauvin
Page : https://labelleassiette.fr/laura-celine-you
Page : https://labelleassiette.fr/boris-banicles
Page : https://labelleassiette.fr/arthur-lefranc
Page : https://labell

In [8]:
df_output.head()

,chef,total,nom,photo,valide,texte_intro,note,avis_nb,a_propos,zone_ville,zone_pays_code,langues,cuisines_types,questions
0,florent-gouverneur,14,Florent Gouverneur,https://assets3.labelleassiette.com/media/prof...,OUI,Amoureux de la gastronomie,4.333333333333333,9,"Après une formation au lycée hôtelier de Metz,...","Lyon, France",FR,\nFrançais\n,"[Bio, Cocktails, Chef étoilé, Brunch, Gastrono...","[Ma cuisine est fonction de la saisonnalité, r..."
1,arnaud-bulliot,16,Arnaud Bulliot,https://assets3.labelleassiette.com/media/prof...,OUI,Les plaisirs de la table à partager !,4.2073170731707314,82,Issue d’une formation hôtelière en région pari...,"Lyon, France",FR,\nFrançais\n,"[Cocktails, Brunch, Gastronomique, Familiale, ...","[Cuisine savoureuse, simple et raffinée, Pour ..."
2,quentin-girerd,2,Quentin Girerd,https://assets3.labelleassiette.com/media/prof...,OUI,Cuisine créative et moderne,4.166666666666667,6,Autodidacte depuis le plus jeune âge le chef a...,"Lyon, France",FR,"\nFrançais,\nEnglish\n","[Cuisine du marché, Bio, Végétarienne, Halal, ...",[Je suis chef à domicile pour faire entrer la ...
3,dario-jarel,12,Dario Jarel,https://assets3.labelleassiette.com/media/prof...,OUI,Passionné de gastronomie Française et Créole.,3.8333333333333335,12,"Dario est originaire de Martinique, c'est en r...","Lyon, France",FR,"\nFrançais,\nEnglish\n","[Cuisine du marché, Bio, Cours de cuisine, Gas...",[Un style de cuisine métissé entre le créole e...
4,la-belle-assiette-france,540,La Belle Assiette France,https://assets3.labelleassiette.com/media/prof...,NON,La meilleure expérience de chef à domicile,0,0,"Pour toute demande spécifique, n'hésitez pas à...",Toute la France,FR,"\nFrançais,\nEnglish\n",[],[]


In [9]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 283
Data columns (total 14 columns):
chef              284 non-null object
total             284 non-null object
nom               284 non-null object
photo             284 non-null object
valide            284 non-null object
texte_intro       284 non-null object
note              284 non-null object
avis_nb           284 non-null object
a_propos          284 non-null object
zone_ville        284 non-null object
zone_pays_code    284 non-null object
langues           284 non-null object
cuisines_types    284 non-null object
questions         284 non-null object
dtypes: object(14)
memory usage: 33.3+ KB


In [13]:
# database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
# table = 'chefs'
# conn = sqlite3.connect(database_file)
# df_output.to_sql(table, conn, if_exists="replace")

In [14]:
df_output.to_excel(OUTPUT_PATH + 'chefs_{}_details.xlsx'.format(date_jour), index=False, header=True)